In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse

from models import AE
from utils import *

# fix seed
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

In [2]:
# make dataset
df = pd.read_csv('./datasets/JNU_elec.csv').set_index('timestamp')
seq_data = create_seq(df, seq_len=24)

# split
train = seq_data[:(365 * 24 * 3)]
val = seq_data[(365 * 24 * 3):(365 * 24 * 4)]
test = seq_data[(365 * 24 * 4):]

print(f'Train: {train.shape}')
print(f'Val: {val.shape}')
print(f'Test: {test.shape}')

Train: (26280, 24, 1)
Val: (8760, 24, 1)
Test: (3601, 24, 1)


In [3]:
# Min-Max normalize
min_val, max_val = np.min(train), np.max(train)

train_norm = minmax(train, min_val, max_val)
val_norm = minmax(val, min_val, max_val)
test_norm = minmax(test, min_val, max_val)

# make tensorflow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_norm, train_norm))
train_dataset = train_dataset.batch(batch_size=64).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_norm, val_norm))
val_dataset = val_dataset.batch(batch_size=64).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_norm, test_norm))
test_dataset = test_dataset.batch(batch_size=64).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [4]:
lstmae = AE()
lstmae.build(input_shape=(None, train.shape[1], train.shape[2]))
lstmae.summary()

Model: "ae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           multiple                  16896     
                                                                 
 decoder (Decoder)           multiple                  115457    
                                                                 
Total params: 132,353
Trainable params: 132,353
Non-trainable params: 0
_________________________________________________________________


In [5]:
gruae = AE(mode='GRU')
gruae.build(input_shape=(None, train.shape[1], train.shape[2]))
gruae.summary()

Model: "ae_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_1 (Encoder)         multiple                  12864     
                                                                 
 decoder_1 (Decoder)         multiple                  91137     
                                                                 
Total params: 104,001
Trainable params: 104,001
Non-trainable params: 0
_________________________________________________________________


In [6]:
tcnae = AE(mode='TCN')
tcnae.build(input_shape=(None, train.shape[1], train.shape[2]))
tcnae.summary()

Model: "ae_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_2 (Encoder)         multiple                  345024    
                                                                 
 decoder_2 (Decoder)         multiple                  189185    
                                                                 
Total params: 534,209
Trainable params: 534,209
Non-trainable params: 0
_________________________________________________________________


In [7]:
from keras import optimizers

lstmae.compile(optimizer=optimizers.Adam(learning_rate=.001), loss='mse')
gruae.compile(optimizer=optimizers.Adam(learning_rate=.001), loss='mse')
tcnae.compile(optimizer=optimizers.Adam(learning_rate=.001), loss='mse')

In [11]:
hist1 = lstmae.fit(train_dataset, epochs=20, batch_size=64, validation_data=(val_dataset))

Epoch 1/20
411/411 [==============================] - 7s 12ms/step - loss: 0.0116 - val_loss: 0.0054
Epoch 2/20
411/411 [==============================] - 4s 10ms/step - loss: 0.0040 - val_loss: 0.0049
Epoch 3/20
411/411 [==============================] - 4s 10ms/step - loss: 0.0032 - val_loss: 0.0046
Epoch 4/20
411/411 [==============================] - 4s 10ms/step - loss: 0.0027 - val_loss: 0.0038
Epoch 5/20
411/411 [==============================] - 4s 10ms/step - loss: 0.0024 - val_loss: 0.0036
Epoch 6/20
411/411 [==============================] - 4s 10ms/step - loss: 0.0023 - val_loss: 0.0034
Epoch 7/20
411/411 [==============================] - 4s 10ms/step - loss: 0.0022 - val_loss: 0.0033
Epoch 8/20
411/411 [==============================] - 4s 10ms/step - loss: 0.0020 - val_loss: 0.0034
Epoch 9/20
411/411 [==============================] - 4s 10ms/step - loss: 0.0019 - val_loss: 0.0026
Epoch 10/20
411/411 [==============================] - 4s 10ms/step - loss: 0.0016 - val_lo

In [14]:
hist1 = gruae.fit(train_dataset, epochs=20, batch_size=64, validation_data=(val_dataset))

Epoch 1/20
411/411 [==============================] - 7s 11ms/step - loss: 0.0232 - val_loss: 0.0187
Epoch 2/20
411/411 [==============================] - 4s 9ms/step - loss: 0.0067 - val_loss: 0.0057
Epoch 3/20
411/411 [==============================] - 4s 9ms/step - loss: 0.0040 - val_loss: 0.0053
Epoch 4/20
411/411 [==============================] - 4s 9ms/step - loss: 0.0037 - val_loss: 0.0049
Epoch 5/20
411/411 [==============================] - 4s 9ms/step - loss: 0.0029 - val_loss: 0.0039
Epoch 6/20
411/411 [==============================] - 4s 9ms/step - loss: 0.0024 - val_loss: 0.0036
Epoch 7/20
411/411 [==============================] - 4s 9ms/step - loss: 0.0024 - val_loss: 0.0035
Epoch 8/20
411/411 [==============================] - 4s 9ms/step - loss: 0.0023 - val_loss: 0.0033
Epoch 9/20
411/411 [==============================] - 4s 9ms/step - loss: 0.0022 - val_loss: 0.0031
Epoch 10/20
411/411 [==============================] - 4s 9ms/step - loss: 0.0021 - val_loss: 0.003

In [15]:
hist1 = tcnae.fit(train_dataset, epochs=20, batch_size=64, validation_data=(val_dataset))

Epoch 1/20
411/411 [==============================] - 5s 11ms/step - loss: 0.0075 - val_loss: 0.0028
Epoch 2/20
411/411 [==============================] - 4s 10ms/step - loss: 0.0014 - val_loss: 0.0016
Epoch 3/20
411/411 [==============================] - 4s 10ms/step - loss: 0.0017 - val_loss: 0.0031
Epoch 4/20
411/411 [==============================] - 4s 10ms/step - loss: 8.3598e-04 - val_loss: 6.7061e-04
Epoch 5/20
411/411 [==============================] - 4s 10ms/step - loss: 4.9442e-04 - val_loss: 7.0306e-04
Epoch 6/20
411/411 [==============================] - 4s 11ms/step - loss: 3.8772e-04 - val_loss: 6.1972e-04
Epoch 7/20
411/411 [==============================] - 4s 11ms/step - loss: 4.1911e-04 - val_loss: 5.5253e-04
Epoch 8/20
411/411 [==============================] - 4s 11ms/step - loss: 4.8310e-04 - val_loss: 4.6695e-04
Epoch 9/20
411/411 [==============================] - 4s 11ms/step - loss: 4.6898e-04 - val_loss: 5.2336e-04
Epoch 10/20
411/411 [======================